In this code we exploit the previous implementation showed in STEP1_LinearDarcy and STEP1_DiffusionTransport in order to solve the cascade problem:

\begin{equation*}
\begin{cases}
 \nu\boldsymbol{u} + \nabla p = \boldsymbol{f} \;\; \text{in} \;\; \Omega ,\\ \text{div} \,  \boldsymbol{u} = 0 \;\; \text{in} \;\; \Omega ,\\ p = p_D\;\; \text{on} \;\; \partial\Omega  
\end{cases}\quad
\begin{cases}
-\kappa\Delta\theta + (\boldsymbol{u}\cdot\nabla)\theta = g \;\; \text{in} \;\; \Omega, \\ \theta = \theta_0 \;\; \text{on} \;\; \partial\Omega
\end{cases}
\end{equation*}

the weak and discrete formulations can be recovered in the mentioned above files where there is also an implementation of them

In [1]:
%%capture
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

In [2]:
from fenics import *
from mshr import *
import matplotlib.pyplot as plt
import numpy as np
import math

In [3]:
# First experiment
nu = Constant(10)
k = Constant(5e-4)

f = Constant((0.0, 0.0))                                        # Source term for Darcy
g = Expression('-4*k-4*(1/nu)*x[0]*x[1]', degree=4, nu=nu, k=k) # Source for advection-diffusion

p_exact = Expression('x[0]*x[1]', degree=2)
u_exact = Expression(('-(1/nu)*x[1]', '-(1/nu)*x[0]'), degree=2, nu=nu)
t_exact = Expression('pow(x[0],2)+pow(x[1],2)', degree=2)

d=100
degree = 2 #r

In [ ]:
# Second experiment
nu = Constant(10)
k = Constant(5e-4)

f = Expression(('nu*pow(sin(x[1]),2)+cos(x[0])*cos(x[1])','nu*pow(cos(x[0]),2)-sin(x[0])*sin(x[1])'), nu=nu, degree=2)
g = Expression('k*2*cos(x[0])*sin(x[1])-sin(x[0])*pow(sin(x[1]),3)+cos(x[1])*pow(cos(x[0]),3)', k=k, degree = 2)

p_exact = Expression('sin(x[0])*cos(x[1])', degree=2)
u_exact = Expression(('pow(sin(x[1]),2)','pow(cos(x[0]),2)'), degree=2)
t_exact = Expression('sin(x[1])*cos(x[0])', degree = 2)

d=100
degree = 2

In [4]:
def DiffAdv_DG(d, degree, k, g, u, t_exact):

  # 1. mesh generation
  mesh = UnitSquareMesh(d, d, 'crossed')

  # 2. definition of finite element space
  V = FunctionSpace(mesh, 'DG', degree)
  W = FunctionSpace(mesh, 'RT', 2)

  # 3. assembling matrices and vectors
  t = TrialFunction(V)
  v = TestFunction(V)
  n = FacetNormal(mesh)

  gamma = 9.1*degree**2
  h = CellDiameter(mesh)
  h_avg = (h('+') + h('-'))/2
  uh = project(u,W)

  a = (k*dot(grad(t) , grad(v)))*dx \
    - (dot(avg(k*grad(t)) , jump(v,n)))*dS \
    - (dot(avg(k*grad(v)) , jump(t,n)))*dS \
    - (dot(k*grad(v) , n*t))*ds \
    + ((gamma/h_avg)*dot(jump(t,n) , jump(v,n)))*dS \
    + (t*v*gamma/h)*ds \
    - (dot(uh*t, grad(v)))*dx \
    + (dot(avg(uh*t) , jump(v,n)))*dS \
    + (abs(dot(uh('+'),n('+')))/2*dot(jump(t,n) , jump(v,n)))*dS \
    + 0.5*dot(uh,n)*t*v*ds \
    - 0.5*div(uh)*t*v*dx

  L = (g*v)*dx \
    - (dot(k*grad(v) , n*t_exact))*ds \
    + (t_exact*v*gamma/h)*ds \
    - 0.5*(dot(uh,n)*t_exact*v)*ds

  th = Function(V)
  solve(a == L, th)

  return th

In [5]:
def solve_DarcyHeatCoupled(d, degree, nu, k, u_exact, p_exact, t_exact, f, g):

    # 1. mesh generation
    mesh = UnitSquareMesh(d, d, 'crossed')

    # 2. definition of a MFE stable coupling
    U = FiniteElement('RT', mesh.ufl_cell(), degree)
    P = FiniteElement('DG', mesh.ufl_cell(), degree-1)
    X = FunctionSpace(mesh, U * P)

    # 3. assembling
    u, p = TrialFunctions(X)
    v, q = TestFunctions(X)
    n = FacetNormal(mesh)

    a = ((nu) * dot(u, v) - q * div(u) - p * div(v)) * dx
    L = dot(f, v) * dx - p_exact * dot(v, n) * ds

    # 4. solve
    x = Function(X)
    solve(a == L, x)
    u, ph = x.split()

    # 5. compute error
    l2err_p = errornorm(p_exact, ph, 'L2')
    l2err_u = errornorm(u_exact, u, 'L2')
    Hdiverr_u = errornorm(u_exact, u, 'Hdiv0')

    # 6. solve the advection-diffusion equation
    th = DiffAdv_DG(d, degree, k, g, u, t_exact)

    l2err_t = errornorm(t_exact, th, 'L2')
    H1err_t = errornorm(t_exact, th, 'H1')

    return u, ph, th, l2err_p, l2err_u, Hdiverr_u, l2err_t, H1err_t

Now we compute the errors and the orders of convergence

In [ ]:
# Compute the errors
error_L2 = np.empty([3,4])
h = np.empty([1,4])
i = 0

for n in [20, 40, 80, 160]:
  u, p, t, l2err_p, l2err_u, Hdiverr_u, l2err_t, H1err_t = solve_DarcyHeatCoupled(n, degree, nu, k, u_exact, p_exact, t_exact, f, g)
  error_L2[0,i] = l2err_u
  error_L2[1,i] = l2err_p
  error_L2[2,i] = l2err_t
  h[0,i] = 1/n
  i = i+1


In [ ]:
print(error_L2)

In [ ]:
# Rescale the errors and plots of the orders of convergence
h = h / h[0,0]
for i in range(3):
  error_L2[i,:] = error_L2[i,:]/error_L2[i,0]

In [ ]:
# Plot the errors
plt.loglog(h[0,:], h[0,:], 'y', label='Order 1')
plt.loglog(h[0,:], h[0,:]**2, 'r', label='Order 2')
plt.loglog(h[0,:], h[0,:]**(1.5), 'k', label='Order 3/2')
plt.loglog(h[0,:], error_L2[0,:], 'c--o', label='L2_u')
plt.loglog(h[0,:], error_L2[1,:], 'm--o', label='L2_p')
plt.loglog(h[0,:], error_L2[2,:], 'g--o', label='L2_t')

plt.title("Orders of convergence cascade problem");
plt.xlabel("h")
plt.ylabel("Errors")
plt.legend(loc=4)

plt.show()

In [ ]:
# Compute the order of convergence
order_L2 =np.empty([3,3])

for j in [1,2,3]:
  order_L2[0,j-1]=math.log(error_L2[0,j-1]/error_L2[0,j],2)#u
  order_L2[1,j-1]=math.log(error_L2[1,j-1]/error_L2[1,j],2)#p
  order_L2[2,j-1]=math.log(error_L2[2,j-1]/error_L2[2,j],2)#t

print(order_L2)